# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

***
*WORK IN PROGRESS*
***

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

### --PENDING-- US County - Alternative: NY Times
- Link: https://github.com/nytimes/covid-19-data/
- **Historic time series data:**
    - US States (cases, deaths) --PENDING--
    - US Counties (cases, deaths) --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-09-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-09-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-10 04:07:00,34.223334,-82.461707,60,0,0,60,"Abbeville, South Carolina, US",244.628369,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-10 04:07:00,30.295065,-92.414197,517,29,0,488,"Acadia, Louisiana, US",833.266178,5.609284
2,51001.0,Accomack,Virginia,US,2020-06-10 04:07:00,37.767072,-75.632346,983,13,0,970,"Accomack, Virginia, US",3041.836861,1.322482
3,16001.0,Ada,Idaho,US,2020-06-10 04:07:00,43.452658,-116.241552,848,22,0,826,"Ada, Idaho, US",176.084487,2.594340
4,19001.0,Adair,Iowa,US,2020-06-10 04:07:00,41.330756,-94.471059,9,0,0,9,"Adair, Iowa, US",125.838926,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9394,9513,9626,9733,9831,9935,10050,10154,10265,10382


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,257,265,270,294,300,309,327,357,369,384
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,33,33,33,33,33,33,34,34,34,34
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,653,661,667,673,681,690,698,707,715,724


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1328,1428,1450,1522,1585,1762,1830,1875,2171,2651
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,872,877,891,898,898,910,925,938,945,960
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,5748,5894,6067,6218,6297,6453,6631,6717,6799,6951


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,172,175,175,177,179,179,179,179,179,180
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,22,22,23,24,26,26,26,27,28,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
507,1025.0,Clarke,Alabama,US,2020-06-10 04:07:00,31.680999,-87.835486,161,3,0,158,"Clarke, Alabama, US",681.568030,1.863354
328,1015.0,Calhoun,Alabama,US,2020-06-10 04:07:00,33.774837,-85.826304,180,3,0,177,"Calhoun, Alabama, US",158.443730,1.666667
1051,1063.0,Greene,Alabama,US,2020-06-10 04:07:00,32.855042,-87.956840,102,5,0,97,"Greene, Alabama, US",1257.551473,4.901961


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,1979411,111989,524855
Brazil,739503,38406,396737
Russia,484630,6134,241917


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,1979411,111989,524855,1342567,2020-06-10 04:07:00,38.236369,-91.403094
Brazil,739503,38406,396737,304360,2020-06-10 04:07:00,-12.669522,-48.480493
Russia,484630,6134,241917,236579,2020-06-10 04:07:00,54.546312,62.120860
United Kingdom,290581,40968,1257,248356,2020-06-10 04:07:00,24.548173,-43.902242
India,276146,7750,134670,133726,2020-06-10 04:07:00,20.593684,78.962880


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,379482,30458,0
New Jersey,164796,12303,0
California,136641,4745,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,379482,30458,0,349024,2020-06-10 04:07:00,42.671593,-75.579694
New Jersey,164796,12303,0,152493,2020-06-10 04:07:00,40.286441,-74.622450
California,136641,4745,0,131896,2020-06-10 04:07:00,37.778248,-120.728666
Illinois,129212,6018,0,123194,2020-06-10 04:07:00,39.845605,-89.165461
Massachusetts,103889,7408,0,96481,2020-06-10 04:07:00,42.176400,-71.577050


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered
Province_State Admin2                                     
New York       New York City     207693   21918          0
Illinois       Cook               83271    4010          0
California     Los Angeles        65945    2710          0
New York       Nassau             40980    2145          0
               Suffolk            40426    1935          0

In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered  Active  \
Province_State Admin2                                                
New York       New York City     207693   21918          0  185775   
Illinois       Cook               83271    4010          0   79261   
California     Los Angeles        65945    2710          0   63235   
New York       Nassau             40980    2145          0   38835   
               Suffolk            40426    1935          0   38491   

                                      Last_Update        Lat        Long  \
Province_State Admin2                                                      
New York       New York City  2020-06-10 04:07:00  40.767273  -73.971526   
Illinois       Cook           2020-06-10 04:07:00  41.841448  -87.816588   
California     Los Angeles    2020-06-10 04:07:00  34.308284 -118.228241   
New York       Nassau         2020-06-10 04:07:00  40.740665  -73.589419   
               Suffolk        2020-06-10 04:07:00  40.883201  -72.801217   

                                 FIPS  
Province_State Admin2                  
New York       New York City  36061.0  
Illinois       Cook           17031.0  
California     Los Angeles     6037.0  
New York       Nassau         36059.0  
               Suffolk        36103.0

##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-07,20342,1246,10154,852,91,26,22794,13130,7265,16902,...,287621,845,4331,2377,331,472,9,484,1089,282
2020-06-08,20917,1263,10265,852,92,26,23620,13325,7267,16968,...,288834,845,4440,2473,332,473,9,496,1200,287
2020-06-09,21459,1299,10382,852,96,26,24761,13675,7274,16979,...,290581,846,4520,2632,332,481,9,524,1200,314


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-07,357,34,707,51,4,3,664,200,102,672,...,40625,23,17,22,0,3,1,112,7,4
2020-06-08,369,34,715,51,4,3,693,211,102,672,...,40680,23,18,22,0,3,1,112,10,4
2020-06-09,384,34,724,51,4,3,717,217,102,672,...,40968,23,18,23,0,3,1,127,10,4


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-07,1875,938,6717,744,24,20,6909,4014,6706,15793,...,1239,730,3354,487,307,403,6,23,912,34
2020-06-08,2171,945,6799,751,38,20,7305,4099,6720,15839,...,1255,738,3357,487,316,404,6,23,912,46
2020-06-09,2651,960,6951,757,38,20,7568,4451,6740,15875,...,1257,754,3459,487,316,404,6,23,912,46


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,693,700,705,705,708,714,730,739,744,752
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,122,122,124,124,124,125,126,126,126,126
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,170,171,174,174,175,179,182,184,187,187
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,82,85,86,87,93,94,96,98,100,104


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-07,20500,543,0,26989,9426,130615,27987,43968,9942,49,...,26493,75408,12066,1063,71,50681,23729,2144,20835,947
2020-06-08,20925,562,0,27678,9740,133816,28169,44092,9972,49,...,27005,76463,12322,1075,71,51251,24041,2161,21038,960
2020-06-09,21422,572,0,28296,10080,136641,28333,44179,10020,49,...,27575,78208,12559,1084,71,51738,24354,2169,21308,970


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-07         265     313     193   77     72     232    449     176   
2020-06-08         272     320     197   79     73     238    455     178   
2020-06-09         282     325     199   85     75     243    464     180   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-07          378       42  ...       2      1       18        3   
2020-06-08          383       42  ...       2      1       18        3   
2020-06-09          391       42  ...       2      1       18        3   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-07             32   100    23          0       39      1  
2020-06-08             33   100    25          0       39      1  
2020-06-09             34   100    35          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-07,692,10,0,1051,154,4632,1527,4071,398,0,...,418,1841,121,55,6,1472,1157,84,647,17
2020-06-08,718,10,0,1053,155,4657,1543,4084,398,0,...,421,1848,124,55,6,1477,1161,84,646,17
2020-06-09,729,11,0,1076,161,4745,1553,4097,410,0,...,435,1867,127,55,6,1496,1176,84,661,17


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-07           5       9       1    1      1       7     22       3   
2020-06-08           5       9       1    1      1       8     24       3   
2020-06-09           5       9       1    1      1       8     24       3   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-07           26        4  ...       0      0        0        0   
2020-06-08           26        4  ...       0      0        0        0   
2020-06-09           26        4  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-07              0     0     0         16        0      0  
2020-06-08              0     0     0         16        0      0  
2020-06-09              0     0     0         16        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  

In [54]:
states_curr = pd.read_csv('https://covidtracking.com/api/states.csv', index_col='state')

In [55]:
states_hist = pd.read_csv('http://covidtracking.com/api/states/daily.csv')

In [56]:
us_curr = pd.read_csv('https://covidtracking.com/api/us.csv')
us_curr

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,lastModified,total,totalTestResults,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20200609,56,1970596,19077587,1661,30547,219089,6450,9141,3090,...,2020-06-09T00:00:00Z,21049844,21048183,21048183,941,1508,395453,17557,413010,b0e06eb05ef18580506e4e035f9f8a77ed714ff7


In [57]:
us_hist = pd.read_csv('https://covidtracking.com/api/us/daily.csv')

### Dataset 3. US Counties - New York Times
https://github.com/nytimes/covid-19-data/

In [58]:
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [59]:
counties.date = pd.to_datetime(counties.date)

In [60]:
nytdate = counties.date.max()
nytdate

Timestamp('2020-06-08 00:00:00')

In [61]:
counties.set_index('date', inplace=True)
counties.head()

,county,state,fips,cases,deaths
date,,,,,
2020-01-21,Snohomish,Washington,53061.0,1,0
2020-01-22,Snohomish,Washington,53061.0,1,0
2020-01-23,Snohomish,Washington,53061.0,1,0
2020-01-24,Cook,Illinois,17031.0,1,0
2020-01-24,Snohomish,Washington,53061.0,1,0


## Data Wrangling

Get the data into a format where it's machine readable for analytics and forecasting

In [62]:
# Covid Tracking Project - US State data.  Look at first 5 rows before reformatting
states_hist.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200609,AK,573.0,66317.0,NaN,12.0,NaN,NaN,NaN,1.0,...,66890,1,0,9eaf1802a047fc56612f95b6e8221d463088d951,0,0,0,0,0,NaN
1,20200609,AL,21422.0,246317.0,NaN,600.0,2087.0,NaN,631.0,NaN,...,267739,11,33,efc4ed6259393a9d1a456d534f048bed440a3188,0,0,0,0,0,NaN
2,20200609,AR,10080.0,155701.0,NaN,173.0,865.0,NaN,NaN,44.0,...,165781,6,21,851214658655b6d73de5bdcef6d60ff6f4cba91f,0,0,0,0,0,NaN
3,20200609,AS,0.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,...,174,0,0,0df19625dbb3d07994abcf96f96cf424a01197f9,0,0,0,0,0,NaN
4,20200609,AZ,28296.0,264917.0,NaN,1243.0,3406.0,438.0,NaN,264.0,...,293213,23,29,13e930e15aa12c07ffcf3af528334c67a01893a1,0,0,0,0,0,NaN


In [63]:
# change YYYYMMDD to YYYY-MM-DD and convert to datetime

states_hist['date'] = states_hist['date'].apply(lambda x : str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:8])
states_hist['date'] = states_hist['date'].apply(lambda x : pd.to_datetime(x) )

In [64]:
# Now set the date as the index

states_hist.set_index('date', inplace=True)
states_hist.sort_index(inplace=True)

In [65]:
states_hist.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,3f9b3067fb6e1b79d174e867e388ae24d90de6e3,0,0,0,0,0,NaN
2020-01-23,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,7b7bb15b16e3613cf1fd2d7cee5a90d483f63662,0,0,0,0,0,NaN
2020-01-24,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,9c9fcbd72b97700fdf886f77e8c16c9538883163,0,0,0,0,0,NaN
2020-01-25,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,9be1bec9296e55b7910716eeef161f40b7dae898,0,0,0,0,0,NaN
2020-01-26,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,c91362aca4975445557b14f49e3ef797de1e0b1c,0,0,0,0,0,NaN


In [66]:
# get the latest date so we can pull recent data as needed
ctp_date = states_hist.index.max()
ctp_date

Timestamp('2020-06-09 00:00:00')

In [67]:
# get a list of the top 15 states using current data
top_states = states_curr['positive'].sort_values(ascending=False).nlargest(15).index
top_states

Index(['NY', 'NJ', 'CA', 'IL', 'MA', 'TX', 'PA', 'FL', 'MI', 'MD', 'GA', 'VA',
       'CT', 'LA', 'OH'],
      dtype='object', name='state')

In [68]:
# Now create a pivot tables for positive, negative, pending, hospitalized, death, total

states_positive = pd.pivot_table(states_hist, index='date', columns='state', values='positive')
states_negative = pd.pivot_table(states_hist, index='date', columns='state', values='negative')
states_pending = pd.pivot_table(states_hist, index='date', columns='state', values='pending')
states_hospitalized = pd.pivot_table(states_hist, index='date', columns='state', values='hospitalized')
states_deaths = pd.pivot_table(states_hist, index='date', columns='state', values='death')
states_tests = pd.pivot_table(states_hist, index='date', columns='state', values='total')

## Start examining the data - Exploratory Data Analysis (EDA)

In [69]:
# Total Global confirmed cases
jh_live_global['Confirmed'].sum()

7242235

In [70]:
# Total Global deaths
jh_live_global['Deaths'].sum()

411436

In [71]:
# Total US confirmed cases
jh_live_global[jh_live_global["Country_Region"]=='US']['Confirmed'].sum()

1979411

In [72]:
# Total US Deaths (per JH dataset)
jh_live_global[jh_live_global["Country_Region"]=='US']['Deaths'].sum()

111989

Make a Pivot Table to summarize the global confirmed cases by Country  
**DONE**  
date_columns = global_confirmed.iloc[:, 4:].columns
confirmed_country = pd.pivot_table(global_confirmed, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
confirmed_country = confirmed_country.transpose()
confirmed_country.index = pd.to_datetime(confirmed_country.index)
confirmed_country = confirmed_country.sort_index()

In [73]:
counties_CA_filter = counties['state']=='California'
counties_CA_cases = pd.pivot_table(counties[counties_CA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [74]:
top_CA_counties = counties_CA_cases.loc[nytdate, :].nlargest(10).index
top_CA_counties

Index(['Los Angeles', 'Riverside', 'San Diego', 'Orange', 'San Bernardino',
       'Alameda', 'Imperial', 'Santa Clara', 'Kern', 'San Francisco'],
      dtype='object', name='county')

In [75]:
counties_MA_filter = counties['state']=='Massachusetts'
counties_MA_cases = pd.pivot_table(counties[counties_MA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [76]:
top_MA_counties = counties_MA_cases.loc[nytdate, :].nlargest(10).index
top_MA_counties

Index(['Middlesex', 'Suffolk', 'Essex', 'Worcester', 'Norfolk', 'Plymouth',
       'Bristol', 'Hampden', 'Barnstable', 'Hampshire'],
      dtype='object', name='county')